In [18]:
import pandas as pd
from env import get_db_url
import numpy as np

# Acquire:

I need to figure out why these are coming out differently after the join. 
- There's a difference of 52,887 records.
    - It looks like the difference here is due to having 52,893 null values in logs.cohort_id.
    - There are 6 cohorts in the cohorts_id table that do not have any server traffic, so perhaps they didn't exist.
    - 52,887 that didn't have anything to tie them to the cohorts table. Probably cohort ids - come back and verify.
- it appears that there there are some IDs and Cohort IDs that do not much.

In [ ]:
## Identifying log entries in 

In [6]:
unknown_traffic = pd.read_sql('''SELECT * FROM logs
LEFT JOIN cohorts
ON logs.cohort_id = cohorts.id
WHERE cohort_id IS NULL
ORDER BY cohort_id;''', url)

In [7]:
unknown_traffic.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2021-01-29,15:24:11,java-iii/passwords,721,None,107.218.181.206,None,None,None,None,None,None,None,None,None
1,2021-01-29,15:24:20,java-iii/sessions-and-cookies,721,None,107.218.181.206,None,None,None,None,None,None,None,None,None
2,2021-01-29,15:24:47,java-iii/servlets,721,None,107.218.181.206,None,None,None,None,None,None,None,None,None
3,2021-01-29,15:41:19,java-iii/passwords,721,None,107.218.181.206,None,None,None,None,None,None,None,None,None
4,2021-01-30,03:18:22,/,723,None,69.58.159.196,None,None,None,None,None,None,None,None,None


### Importing from CSV:

In [93]:
df1 = pd.read_csv('anonymized_curriculum_access.csv', sep = ' ')

In [94]:
df1.shape[0]

900223

In [95]:
df1

,date,time,path,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1.0,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1.0,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1.0,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1.0,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2.0,22.0,97.105.19.61
...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64.0,28.0,71.150.217.33
900219,2021-04-21,16:42:02,jquery/mapbox-api,64.0,28.0,71.150.217.33
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64.0,28.0,71.150.217.33
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744.0,28.0,24.160.137.86


## Importing from the database using SQL

### Using Default Join:

In [26]:
url = get_db_url('curriculum_logs')

query = '''SELECT * FROM logs
JOIN cohorts 
ON logs.cohort_id = cohorts.id;'''

In [27]:
df2 = pd.read_sql(query, url)

In [28]:
df2.shape[0]

847330

In [29]:
df2.isnull().sum()

date               0
time               0
path               1
user_id            0
cohort_id          0
ip                 0
id                 0
name               0
slack              0
start_date         0
end_date           0
created_at         0
updated_at         0
deleted_at    847330
program_id         0
dtype: int64

### Using Right Join:

In [96]:
url = get_db_url('curriculum_logs')

query = '''SELECT * FROM logs
RIGHT JOIN cohorts 
ON logs.cohort_id = cohorts.id;'''

In [97]:
df3 = pd.read_sql(query, url)

In [60]:
df3.shape[0]

847336

In [88]:
df3.isnull().sum()

date               6
time               6
path               7
user_id            6
cohort_id          6
ip                 6
id                 0
name               0
slack              0
start_date         0
end_date           0
created_at         0
updated_at         0
deleted_at    847336
program_id         0
dtype: int64

In [91]:
df3[df3.cohort_id.isnull()]

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
3038,None,None,None,NaN,NaN,None,30,Ada,#ada,2019-02-04,2019-06-16,2019-02-04 20:13:32,2019-02-04 20:13:32,None,3
3039,None,None,None,NaN,NaN,None,60,Io,#io,2020-07-20,2021-01-21,2020-07-20 13:24:55,2020-09-30 15:54:53,None,2
164848,None,None,None,NaN,NaN,None,136,Placeholder for students in transition,#null,2021-03-03,2029-03-01,2021-03-03 21:50:49,2021-03-03 21:52:56,None,2
164854,None,None,None,NaN,NaN,None,20,Redwood,#redwood,2017-07-31,2017-12-06,2017-07-31 19:34:11,2017-08-08 21:59:33,None,2
436200,None,None,None,NaN,NaN,None,3,Carlsbad,#carlsbad,2014-09-04,2014-11-05,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
507978,None,None,None,NaN,NaN,None,10,Balboa,#balboa,2015-11-03,2016-03-11,2016-06-14 19:52:26,2016-06-14 19:52:26,None,4


### Using Left Join:

In [2]:
url = get_db_url('curriculum_logs')

query = '''SELECT * FROM logs
LEFT JOIN cohorts 
ON logs.cohort_id = cohorts.id;'''

In [3]:
df4 = pd.read_sql(query, url)

In [4]:
df4.shape[0]

900223

In [5]:
df4.isnull().sum()

date               0
time               0
path               1
user_id            0
cohort_id      52893
ip                 0
id             52893
name           52893
slack          52893
start_date     52893
end_date       52893
created_at     52893
updated_at     52893
deleted_at    900223
program_id     52893
dtype: int64

## Trying to understand difference in counts:

In [25]:
df2.shape[0] - df3.shape[0]

NameError: name 'df2' is not defined

In [80]:
df2.isnull().sum()

date               0
time               0
path               1
user_id            0
cohort_id      52893
ip                 0
id             52893
name           52893
slack          52893
start_date     52893
end_date       52893
created_at     52893
updated_at     52893
deleted_at    900223
program_id     52893
dtype: int64

In [82]:
df2[df2.cohort_id.isnull()]

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
411,2018-01-26,16:46:16,/,48,NaN,97.105.19.61,NaN,None,None,None,None,None,None,None,NaN
412,2018-01-26,16:46:24,spring/extra-features/form-validation,48,NaN,97.105.19.61,NaN,None,None,None,None,None,None,None,NaN
425,2018-01-26,17:54:24,/,48,NaN,97.105.19.61,NaN,None,None,None,None,None,None,None,NaN
435,2018-01-26,18:32:03,/,48,NaN,97.105.19.61,NaN,None,None,None,None,None,None,None,NaN
436,2018-01-26,18:32:17,mysql/relationships/joins,48,NaN,97.105.19.61,NaN,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899897,2021-04-21,12:49:00,javascript-ii,717,NaN,136.50.102.126,NaN,None,None,None,None,None,None,None,NaN
899898,2021-04-21,12:49:02,javascript-ii/es6,717,NaN,136.50.102.126,NaN,None,None,None,None,None,None,None,NaN
899899,2021-04-21,12:51:27,javascript-ii/map-filter-reduce,717,NaN,136.50.102.126,NaN,None,None,None,None,None,None,None,NaN
899900,2021-04-21,12:52:37,javascript-ii/promises,717,NaN,136.50.102.126,NaN,None,None,None,None,None,None,None,NaN


## I'm mad:

## cohort_id's in logs table:

In [10]:
cid = pd.read_sql('''SELECT DISTINCT(cohort_id)
FROM logs
ORDER BY (cohort_id);''', url)

<bound method IndexOpsMixin.value_counts of 0       NaN
1       1.0
2       2.0
3       4.0
4       5.0
5       6.0
6       7.0
7       8.0
8       9.0
9      11.0
10     12.0
11     13.0
12     14.0
13     15.0
14     16.0
15     17.0
16     18.0
17     19.0
18     21.0
19     22.0
20     23.0
21     24.0
22     25.0
23     26.0
24     27.0
25     28.0
26     29.0
27     31.0
28     32.0
29     33.0
30     34.0
31     51.0
32     52.0
33     53.0
34     55.0
35     56.0
36     57.0
37     58.0
38     59.0
39     61.0
40     62.0
41    132.0
42    133.0
43    134.0
44    135.0
45    137.0
46    138.0
47    139.0
Name: cohort_id, dtype: float64>

In [24]:
cid = pd.read_sql('''SELECT cohort_id, COUNT(cohort_id)
FROM logs
GROUP BY (cohort_id);''', url)

cid

,cohort_id,COUNT(cohort_id)
0,8.0,1712
1,22.0,30926
2,21.0,7444
3,1.0,8890
4,16.0,755
5,18.0,2158
6,19.0,1237
7,NaN,0
8,13.0,2845
9,14.0,9587


pandas.core.frame.DataFrame

In [12]:
cid.cohort_id.unique()

array([ nan,   1.,   2.,   4.,   5.,   6.,   7.,   8.,   9.,  11.,  12.,
        13.,  14.,  15.,  16.,  17.,  18.,  19.,  21.,  22.,  23.,  24.,
        25.,  26.,  27.,  28.,  29.,  31.,  32.,  33.,  34.,  51.,  52.,
        53.,  55.,  56.,  57.,  58.,  59.,  61.,  62., 132., 133., 134.,
       135., 137., 138., 139.])

In [22]:
cid[cid.cohort_id == np.nan]

,cohort_id
